In [3]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()

spark = SparkSession.builder.appName("Practice-itv021172-2 ").config('spark.ui.port','0').config('spark.sql.warehouse.dir',f"/user/itv021172/warehouse").enableHiveSupport().master('yarn').getOrCreate()

In [4]:
schema ='''loan_id string,total_principal_recieved float,total_interest_recievd float,total_late_fee_recieved float,
total_payment_recieved float,last_payment_amount float,last_payement_date string,next_payment_date string'''

In [5]:
df = spark.read.format('csv').option('header',True).schema(schema).load('/public/trendytech/lendingclubproject/raw/loans_repayments_csv/part-00000-1dde4854-b613-4f8a-abf8-c50a004013a5-c000.csv')

In [6]:
from pyspark.sql.functions import *

In [7]:
loan_repayment = df.withColumn('Ingest_date',current_timestamp())

In [27]:
filterdata = loan_repayment.dropna(subset=['total_principal_recieved','total_interest_recievd','total_late_fee_recieved','total_payment_recieved','last_payment_amount','last_payement_date'])

In [ ]:
filterdata

In [18]:
filldata = filterdata.withColumn('total_payment_recieved',when((col('total_payment_recieved') == 0) &
                                                               (col('total_principal_recieved') != 0),
                                                               col('total_interest_recievd') +col('total_late_fee_recieved') +  col('total_principal_recieved'))
                                 .otherwise(col('total_payment_recieved')))

In [20]:
loanrepadata = filldata.filter('total_payment_recieved != 0')

In [23]:
lastpyamentfix = loanrepadata.withColumn('last_payement_date', when(col('last_payement_date')==0.0 ,None).otherwise(col('last_payement_date')))

In [28]:
finalloanReapymentddata =lastpyamentfix.withColumn('next_payment_date', when(col('next_payment_date')==0.0 ,None).otherwise(col('next_payment_date')))